# Methods for achieving expert agreement of clinical images: the Medical Annotation and Review of Superpixels (MARk-SUP)
## Web-based expert annotation platform piloted for expert annotation and quantitative analysis of dermoscopic features

### *Online supplementary Jupyter notebook with analyses*

The cells in this notebook can be run in succession, or, as
desired, cell-by-cell.

**Please note:** the first two cells **MUST** be run in order
to establish the necessary variables in the workspace of the
python kernel!

In [ ]:
# imports
import os
from matplotlib import pyplot
%matplotlib inline
import imageio
import numpy
from isicarchive import func
from isicarchive.api import IsicApi

# function for mean and sample STD
def mean_std(a:list, is_sample:bool=True):
    ddof = 1 if is_sample else 0
    return (numpy.mean(a), numpy.std(a, ddof=ddof))

# set to True if you would like (more) details in the print-out
print_details = False
print_fine_details = False
heatmap_mix_colors = False
heatmap_underlay_gray = 0.8
heatmap_resize_output = 4096
heatmap_legend_font_size = 144.0

# please change the username accordingly!
username = 'weberj3@mskcc.org'

# root folder for all ISIC related data
doc_folder = 'Z:\\10.Imaging Informatics\\'

# cache folder
cache_folder = doc_folder + 'ISIC' + os.sep + 'cache'

# show URL requests (for debugging purposes only!)
debug = False

# instantiate API object
api = IsicApi(username, cache_folder=cache_folder, debug=debug)

In [ ]:
# study folder
study_folder = doc_folder + 'EASY' + os.sep + 'PILOT' + os.sep

# load study and data
study = api.study('ISIC Annotation Study - All Features')
study.cache_image_data()
study.load_annotations()

# load meta data
meta_data_url = ('https://raw.githubusercontent.com/neuroelf/' +
    'isicarchive/master/data/EASY_pilot_diagnoses.csv')
study.load_meta_data(meta_data_url, list_to_dict=True,
    dict_key='name', extract_key=['diagnosis', 'exemplar'])

# and create a dictionary mapping diagnosis to a list of images
diag_images = dict()
for (name, diag) in study.meta_data['diagnosis'].items():
    if not diag in diag_images:
        diag_images[diag] = []
    diag_images[diag].append(name)

# same for exemplar features
exem_images = dict()
for (name, exemplar) in study.meta_data['exemplar'].items():
    if not exemplar:
        continue
    if not exemplar in exem_images:
        exem_images[exemplar] = []
    exem_images[exemplar].append(name)

# select only from users that completed the study
num_images = len(study.images)
users = [u for (u,c) in study.user_completion.items() if c==num_images]

# create heatmaps with default settings (if not yet done)
study_stats_file = study_folder + 'heatmap_stats.json.gz'
if not os.path.exists(study_stats_file):
    study_stats = study.image_heatmaps(study_folder, users=users,
        mix_colors=heatmap_mix_colors, underlay_gray=heatmap_underlay_gray, 
        resize_output=heatmap_resize_output, font_size=heatmap_legend_font_size)
else:
    study_stats = func.gzip_load_var(study_stats_file)

# select those annotations, and gather basic statistics
study.select_annotations(users=users)
total_features_annotations = sum(
    [len(a.features) for a in study.annotation_selection.values()])
selected_features = dict()
for annotation in study.annotation_selection.values():
    for feature in annotation.features:
        selected_features[feature] = True

This gives some basic statistics of the study:

In [ ]:
print('{1:d} readers annotated the {2:d} dermoscopic images.'.format(
    study.name, len(users), len(study.images)))
if print_details:
    print(' ... per diagnosis:')
    for (diag, image_list) in diag_images.items():
        print(' - {0:d} in {1:s}'.format(len(image_list), diag))

In [ ]:
print('Out of {0:d} offered features, {1:d} were selected at least once.'.format(
    len(study.features), len(selected_features)))

In [ ]:
print('Each reader annotated an average of {0:.1f} features per lesion.'.format(
    float(total_features_annotations) / len(study.annotation_selection)))
print('In total, {0:d} feature annotations (markups) were made.'.format(
    total_features_annotations))

Some additional computations for complex statistics:

In [ ]:
# compute minimum and maximum number of features (per image/diagnosis)
(fmin, fmin_std, fmin_diag) = (85.0, 0.0, None)
(fmax, fmax_std, fmax_diag) = (0.0, 0.0, None)
if print_details:
    print('On average, images with diagnosis ...')
for diagnosis in sorted(diag_images.keys()):
    study.select_annotations(images=diag_images[diagnosis], users=users)
    ao = [a for a in study.annotation_selection.values()]
    fn = [None] * len(ao)
    for (idx,a) in enumerate(ao):
        fn[idx] = len(a.features)
    (m,s) = mean_std(fn)
    if m < fmin:
        (fmin, fmin_std, fmin_diag) = (m, s, diagnosis)
    if m > fmax:
        (fmax, fmax_std, fmax_diag) = (m, s, diagnosis)
    if print_details:
        print(' - "{0:s}" have {1:.2f} ± {2:.2f} annotations.'.format(
            diagnosis, m, s))
print('Based on the diagnosis, the number of features per image varied from')
print('{0:.2f} (±{1:.2f}) for {2:s} to {3:.2f} (±{4:.2f}) for {5:s}.'.format(
    fmin, fmin_std, fmin_diag, fmax, fmax_std, fmax_diag))

In [ ]:
# for each image, test whether all five raters agreed on one feature
image_agreed = [False] * len(study.images)
image_agreed_features = [[] for l in range(len(study.images))]
image_orphan_features = []
for (idx, image) in enumerate(study.images):
    image_id = image['_id']
    study.select_annotations(images=[image_id], users=users)
    ao = [a for a in study.annotation_selection.values()]
    for feature in ao[0].features.keys():
        agreed = [False] * len(ao)
        feature_syns = api.feature_synonyms(feature)
        for (aidx, a) in enumerate(ao):
            for f in feature_syns:
                if f in a.features:
                    agreed[aidx] = True
                    break
        if all(agreed):
            image_agreed[idx] = True
            image_agreed_features[idx].append(feature)
    for (aidx1, a1) in enumerate(ao):
        for f1 in a1.features.keys():
            feature_syns = api.feature_synonyms(feature)
total_agreements = 0
if print_details:
    print('Feature-in-image agreements:')
for (image, a, af) in zip(study.images, image_agreed, image_agreed_features):
    if a:
        total_agreements += len(af)
        image_name = image['name']
        image_diag = study.meta_data['diagnosis'][image_name]
        if print_details:
            print(' - {0:s} ({1:s}): {2:s}'.format(image_name,
                image_diag, ', '.join(af)))
print('There were a total of {0:d} feature-in-image agreements.'.format(
    total_agreements))
print(('These were reached in a total of {0:d} images.').format(
    numpy.sum(image_agreed)))

In [ ]:
# count orphan features
orphans = 0
orphan_image_features = []
orphan_features = dict()
for (idx, image) in enumerate(study.images):
    image_id = image['_id']
    image_name = image['name']
    study.select_annotations(images=[image_id], users=users)
    ao = [a for a in study.annotation_selection.values()]
    for (aidx, a) in enumerate(ao):
        for feature in a.features:
            is_orphan = True
            feature_syns = api.feature_synonyms(feature)
            for (aidx2, a2) in enumerate(ao):
                if aidx == aidx2:
                    continue
                for feature2 in a2.features:
                    if feature2 in feature_syns:
                        is_orphan = False
                        break
                if not is_orphan:
                    break
            if is_orphan:
                orphans += 1
                orphan_image_features.append(feature + ' in ' + 
                    image_name + ' by ' + a.user['name'])
                if not feature_syns[0] in orphan_features:
                    orphan_features[feature_syns[0]] = 0
                orphan_features[feature_syns[0]] += 1
print('An orphan observation of a feature occurred {0:d} times.'.format(
    orphans))
if print_details:
    print('{0:d} orphan features were selected:'.format(orphans))
    for orphaned in orphan_image_features:
        print(' - ' + orphaned)
    print('Presenting by list of features (collapsing synonyms):')
    for feature in sorted(orphan_features.keys()):
        print(' - {0:-2d} times "{1:s}"'.format(
            orphan_features[feature], feature))

In [ ]:
# how many exemplar feature annotations for each image with an exemplar?
total_exemplar_images = 0
total_found_direct = 0
total_found_list = []
total_found_category = 0
total_found_specific = 0
for exemplar in sorted(exem_images.keys()):
    images = exem_images[exemplar]
    if print_details:
        print('Exemplar "{0:s}" with {1:d} images:'.format(
            exemplar, len(images)))
    for image in images:
        total_exemplar_images += 1
        imag_diag = study.meta_data['diagnosis'][image]
        study.select_annotations(images=[image], users=users)
        ao = [a for a in study.annotation_selection.values()]
        found_direct = 0
        found_category = 0
        found_specific = 0
        for (idx,a) in enumerate(ao):
            if exemplar in a.features:
                found_direct += 1
                found_category += 1
                found_specific +=1
                continue
            found_at_all = False
            for feature in a.features:
                feature = feature.split(' : ')
                if exemplar[0:len(feature[0])] == feature[0]:
                    found_category += 1
                    found_at_all = True
                if feature[-1] in exemplar:
                    found_specific += 1
                    found_at_all = True
                if found_at_all:
                    break
        if found_direct == len(ao):
            total_found_direct += 1
            total_found_list.append(image)
        if found_specific == len(ao):
            total_found_specific += 1
        if found_category == len(ao):
            total_found_category += 1
        if print_details:
            print((' - {0:s} ({1:s}) has {2:d} annotations; ' +
                   '{3:d}, {4:d}, and {5:d} with the full, category, ' +
                   'and specific exemplar').format(image, imag_diag,
                    len(ao), found_direct, found_category, found_specific))
print('Out of {0:d} images with an exemplar, all readers identified this'.format(
    total_exemplar_images))
print('feature {0:d} times, with {1:d} category and {2:d} specific hits.'.format(
    total_found_direct, total_found_category, total_found_specific))
if print_details:
    print('Direct hits:')
    for image in total_found_list:
        print(' - {0:s} ({1:s}, {2:s})'.format(image,
            study.meta_data['diagnosis'][image],
            study.meta_data['exemplar'][image]))

In [ ]:
# compute the superpixel-wise agreement
# (i.e. where 2 or more readers all agreed or any reader
# disagreed with the feature in the same superpixel)
if print_details:
    print('Per-image super-pixel agreement:')
image_sp_stats = dict()
total_agreed = 0
total_disagreed = 0
sp_agreed_features = dict()
for (image_name, image_stats) in study_stats.items():
    orphans = 0
    agreed = 0
    disagreed = 0
    for (spidx, sp_stats) in image_stats['sp'].items():
        sp_keys = list(sp_stats.keys())
        for sp_key in sp_keys:
            if not sp_key in sp_agreed_features:
                sp_agreed_features[sp_key] = {
                    'agreed': 0, 'disagreed': 0, 'overlap': dict()
                }
        if len(sp_keys) == 1:
            if len(sp_stats[sp_keys[0]]) == 1:
                orphans += 1
            else:
                agreed += 1
                sp_agreed_features[sp_keys[0]]['agreed'] += 1
        else:
            disagreed += 1
            for sp_key in sp_keys:
                sp_agreed_features[sp_key]['disagreed'] += 1
                for o_key in sp_keys:
                    if sp_key != o_key:
                        if not o_key in sp_agreed_features[sp_key]['overlap']:
                            sp_agreed_features[sp_key]['overlap'][o_key] = 0
                        sp_agreed_features[sp_key]['overlap'][o_key] += 1
    total_agreed += agreed
    total_disagreed += disagreed
    image_sp_stats[image_name] = {
        'orphans': orphans, 'agreed': agreed, 'disagreed': disagreed}
    if print_details:
        print(' - {0:s} {1:-4.1f}% agreed ({2:d} SPs, w/o orphans; {3:s})'.format(
            image_name, 100.0 * float(agreed) / float(agreed + disagreed),
            agreed + disagreed, study.meta_data['diagnosis'][image_name]))
print('Total agreement {0:-4.1f}% ({1:d} out of {2:d} total superpixels)'.format(
    100.0 * float(total_agreed) / float(total_agreed + total_disagreed),
    total_agreed, total_agreed + total_disagreed))
if print_details:
    print('Agreement by feature:')
    for sp_key in sorted(sp_agreed_features.keys()):
        sp_agreement = sp_agreed_features[sp_key]
        print(' - {0:-4.1f}% agreement for {1:s} ({2:d} SPs)'.format(
            100.0 * float(sp_agreement['agreed']) /
            float(sp_agreement['agreed'] + sp_agreement['disagreed']),
            sp_key, sp_agreement['agreed'] + sp_agreement['disagreed']))
        if print_fine_details:
            for o_key in sorted(sp_agreement['overlap'].keys()):
                print('   - {0:-3d} overlapped with {1:s}'.format(
                    sp_agreement['overlap'][o_key], o_key))

## Heatmaps

The images below show (1) a demonstration of people using
different terms for the same feature, (2) people agreeing
agreeing on one specific term (but not others), and (3)
everybody agreeing on the (singular) term in an image.

In [ ]:
# show two images (heatmaps) of the study with agreement examples
study_folder = doc_folder + 'EASY' + os.sep + 'PILOT' + os.sep
print('(1)')
api.show_image_in_notebook(
    api.read_image(study_folder + 'ISIC_0016094.jpg'),
    figsize=(15,15))

In [ ]:
print('(2)')
api.show_image_in_notebook(
    api.read_image(study_folder + 'ISIC_0016128.jpg'),
    figsize=(15,15))

In [ ]:
print('(3)')
api.show_image_in_notebook(
    api.read_image(study_folder + 'ISIC_0015549.jpg'),
    figsize=(15,15))

In [ ]:
mix_colors = False
underlay_gray = 0.8
study_stats = study.image_heatmaps(study_folder,
    image_sel=['name', 'in', ['ISIC_0015549','ISIC_0016094','ISIC_0016128']], users=users,
    mix_colors=mix_colors, underlay_gray=underlay_gray, resize_output=4096, font_size=144.0)